In [ ]:
# Generate linked scatterplot + histogram
# Minimal working example
import numpy as np
import math 
from bokeh.layouts import column, row, grid
from bokeh.models import ColumnDataSource, CustomJS, Button, HoverTool
from bokeh.plotting import figure, show
from random import random
from bokeh.io import show

# Screen
source = ColumnDataSource(data=dict(x=[], y=[]))
max_x = 10

p = figure(
    title="Add 1 particle on click or send 100 particles at once",
    tools="pan,wheel_zoom,box_zoom,reset",     
    sizing_mode="stretch_width", 
    width=300,
    height=300,
    x_axis_label="x",
    y_axis_label="y",
    x_range=(0,max_x),
    y_range=(0, 1),
)

p.circle('x', 'y', source=source, size=10, fill_color="red")

# Probability density + hisogram
pd = figure(
    plot_width=300,
    plot_height=300,
    x_range=(0, max_x),
    title="Watch Histogram Here",
)

# rng = np.random.default_rng()  # generator of random ?
# random = rng.normal(loc=0, scale=1, size=1000)  # array of random variables from normal dist.: loc-mean, scale-var
Nbins = 10 # počet binů
hist, bins = np.histogram([], range=(0, max_x), bins=Nbins) # hist = rel. četnosti, bins = array dělicích bodů binů
source_hist = ColumnDataSource(data=dict(hist=hist, bins=bins[:-1]))
pd.vbar(top="hist", x="bins", width=max_x/Nbins, source=source_hist, alpha=0.6, color="firebrick")


# Buttons ----------------------------------------------------------------------
# Send 100 particles at once
button100 = Button(label="Send 100 particles", button_type="success")
# -----------------------------------------------------------------------------
# JS CALLBACKS

js_addtohist = """
    function addtohist() {
        let bhist = s_hist.data.bins;            // bhist = [ 0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9 ]
        let interval = bhist[1] - bhist[0];
        let new_x = Math.random();
        source.data.x.push(new_x)
        source.data.y.push(Math.random())
        let j = Math.floor((new_x - bhist[0])/interval);  // index of bin where a particle with coord. new_x is added
        s_hist.data.hist[j]++;
        
        // --------------> dodělat normalizaci na rel. četnosti
        // normalization; number of particles = source.data.x.length
       // for (let i = 0; i < bhist.length - 1; i++) {
       //     s_hist.data.hist[i] = s_hist.data.hist[]/source.data.x.length;
       // }
        
    // update the data source with local changes
    source.change.emit();
    s_hist.change.emit();
    }
"""

callback2 = CustomJS(args=dict(source=source, s_hist=source_hist), code="""
    
    let bhist = s_hist.data.bins;
    let interval = bhist[1] - bhist[0];
    for (let i = 0; i < 100; i++) {
        let new_x = Math.random();
        source.data.x.push(new_x)
        source.data.y.push(Math.random())
        let j = Math.floor((new_x - bhist[0])/interval);
        s_hist.data.hist[j]++;
    }
    
    // update the data source with local changes
    source.change.emit();
    s_hist.change.emit();
     """)

callback1 = CustomJS(args=dict(source=source, s_hist=source_hist), code="""

    let bhist = s_hist.data.bins;            // bhist = [ 0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9 ]
    let interval = bhist[1] - bhist[0];
        let new_x = Math.random();
        source.data.x.push(new_x)
        source.data.y.push(Math.random())
        let j = Math.floor((new_x - bhist[0])/interval);  // index of bin where a particle with coord. new_x is added
        s_hist.data.hist[j]++;
        
        // --------------> dodělat normalizaci na rel. četnosti
        // normalization; number of particles = source.data.x.length
       // for (let i = 0; i < bhist.length - 1; i++) {
       //     s_hist.data.hist[i] = s_hist.data.hist[]/source.data.x.length;
       // }
        
    // update the data source with local changes
    source.change.emit();
    s_hist.change.emit();
     """)

callback = CustomJS(args=dict(source=source, s_hist=source_hist), code=js_addtohist+"""
    addtohist();
    """)

# -------------------------------------------------------------------------------
# Callbacks to objects

button100.js_on_event('button_click', callback1)

show(column(p, pd, button100))

In [ ]:
np.histogram?

In [ ]:
callback2 = CustomJS(args=dict(source=source, s3=source_hist), code="""
    
    let i = 0;
    for (let i = 0; i < 100; i++) {
    source.data.x.push(Math.random())
    source.data.y.push(Math.random()) 
    }
    
    // update the data source with local changes
    source.change.emit()
    
   // Histogram actualisation
        var dh = source.data.x;
        
        // get the histogram numbins, bin width from initial histogram
        var bins = [];
        var bhist = s3.data["bins"];
        var interval = bhist[1] - bhist[0];
        
        //Setup Bins
        for(let i = 0; i < bhist.length; i++){
          bins.push(0)
        }
        //Loop through data and add to bin's count
        for (let i = 0; i < dh.length; i++){
          let j = Math.floor((dh[i] - bhist[0])/interval);
          bins[j]++;
          }  
        // push the bin values back into the original histogram]]]}
        s3.data.hist = [];
        for (let j = 0; j < bins.length; j++) {
            s3.data.hist.push(bins[j])
        }
        s3.change.emit();
     """)

In [ ]:
len(hist)

In [ ]:
# Histogram in Bokeh
import numpy as np

from bokeh.plotting import figure, show

rng = np.random.default_rng()
x = rng.normal(loc=0, scale=1, size=1000)

plot = figure(width=670, height=400, toolbar_location=None,
           title="Normal (Gaussian) Distribution")

# Histogram
bins = np.linspace(-3, 3, 40)
hist, edges = np.histogram(x, density=False, bins=bins)
plot.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
         fill_color="skyblue", line_color="white",
         legend_label="1000 random samples")

# Probability density function
x = np.linspace(-3.0, 3.0, 100)
pdf = np.exp(-0.5*x**2) / np.sqrt(2.0*np.pi)
plot.line(x, pdf, line_width=2, line_color="navy", legend_label="Probability Density Function")

plot.y_range.start = 0
plot.xaxis.axis_label = "x"
plot.yaxis.axis_label = "PDF(x)"

show(plot)